In [1]:
# Husayn El Sharif

In [2]:
# imports
from pyspark.sql import SparkSession

from pyspark.sql.functions import regexp_replace, col, max, min, avg, year



In [3]:
# Start Session
spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/25 15:16:53 WARN Utils: Your hostname, Husayn-SLS2, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/25 15:16:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/25 15:16:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
listings = spark.read.csv("airbnb_data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)

listings.count()

16822

In [5]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [6]:
# 1. Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL

picture_urls = (
    listings
    .select('picture_url')
    .filter(listings['picture_url'].isNotNull())
)

picture_urls.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------+
|picture_url                                                                                              |
+---------------------------------------------------------------------------------------------------------+
|https://a0.muscache.com/pictures/miso/Hosting-57818/original/4504c3d4-931e-4f25-ba08-b9f7dfe98ecb.jpeg   |
|https://a0.muscache.com/pictures/443102/f6b94ead_original.jpg                                            |
|https://a0.muscache.com/pictures/8c21bda0-5194-494b-acbc-9533929167b7.jpg                                |
|https://a0.muscache.com/pictures/828393/baabb60f_original.jpg                                            |
|https://a0.muscache.com/pictures/1251401/830042de_original.jpg                                           |
|https://a0.muscache.com/pictures/airflow/Hosting-216046/original/8809c48a-30db-4baf-bdfc-a791e8bf4294.jpg|
|https://a0.muscache.com/pic

In [7]:
# 2. Get number of properties that get more than 10 reviews per month
ten_plus_reviews = (
    listings
    .filter(listings['reviews_per_month'] > 10)
)

ten_plus_reviews.count()

43

In [8]:
# 3. Get properties that have more bathrooms than bedrooms
more_baths_than_beds = (
    listings
    .select(['id', 'name', 'bedrooms', 'bathrooms'])
    .where(listings['bathrooms'] > listings['bedrooms'])
)

more_baths_than_beds.show(10, truncate=False)
more_baths_than_beds.count()

+-------+-------------------------------------+--------+---------+
|id     |name                                 |bedrooms|bathrooms|
+-------+-------------------------------------+--------+---------+
|129099 |MIAMI - AMAZING APARTMENT OCEANVIEW  |1       |1.5      |
|217990 |COSY APT OVER THE BEACH              |1       |1.5      |
|325734 |Hidden Gem for two                   |0       |1.0      |
|1305586|Oceanfront 1 Bedroom 2 Bath Penthouse|1       |1.5      |
|1548483|"Welcome home on Hallandale Beach"   |1       |1.5      |
|1646485|Amazing location across the beach!   |3       |3.5      |
|1744871|Oceanfront condo in Fort Lauderdale  |0       |1.0      |
|2011721|3 - Beach Bungalow w/ Free Parking   |0       |1.0      |
|2022117|Furnished Room # 3 share bathroom    |1       |2.0      |
|2028061|Modern, quaint, fully renovated      |1       |1.5      |
+-------+-------------------------------------+--------+---------+
only showing top 10 rows


1418

In [9]:
# 4. Get properties where the price is greater than 5,000. Collect the result as a Python list
# Remember to convert a price into a number first!

# check type of price column
listings.schema["price"].dataType # price is a string!

# create numeric price column
listings_with_numeric_price = listings.withColumn("price_numeric", regexp_replace(col("price"), "[$,]", "").cast("double") )

result = (
    listings_with_numeric_price
    .where((listings_with_numeric_price['price_numeric'] > 5000) & (listings_with_numeric_price['name'].isNotNull()) )
    .select(['name', 'price_numeric'])
).toPandas()

result

,name,price_numeric
0,"SKY PRIVATE APARTMENT FOR 2, WITH POOL ACCESS!",5955.0
1,BEST ROOM FOR 2/3 IN POOL HOUSE BY AIR/CRUISE ...,5153.0
2,LUXURY MIAMI HALLANDALE HOLLYWOOD APART..,6000.0
3,AmazingViews 5BR 2Corner ComboUnits @Hyde -26f...,5184.0
4,Enjoy Peace & tranquillity. A home away from home,95000.0
...,...,...
78,Hillsboro Beach Resort by Kasa | Oceanfront Suite,10019.0
79,Entertainment Paradise,16429.0
80,4 Ocean-Inspired Units with Outdoor Pool,40000.0
81,Waterfront Oasis Near Las Olas – Walk to Beach,7989.0


In [10]:
# 5. Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5
# Consider using the "&" operator

listings_with_numeric_price = listings.withColumn("price_numeric", regexp_replace(col("price"), "[$,]", "").cast("double") )

result = (
    listings_with_numeric_price
    .where(
        (listings_with_numeric_price['price_numeric'] < 150) & 
        (listings_with_numeric_price['number_of_reviews'] > 20) &
        (listings_with_numeric_price['review_scores_rating'] > 4.5)
        )
    .select(['name', 'price_numeric', 'number_of_reviews', 'review_scores_rating'])
    .dropna()
).toPandas()

result

,name,price_numeric,number_of_reviews,review_scores_rating
0,Country Living for Four,79.0,298,4.85
1,Fabulous & Private 475 sq ft Suite,75.0,49,4.96
2,Hidden Gem for two,43.0,253,4.80
3,Tropical Guest House - Heart of Ft. Lauderdale!,74.0,62,4.82
4,Charming Comfort... Ft Lauderdale Home,82.0,40,4.85
...,...,...,...,...
3186,Under the Sea Studio | Beach Steps Away,97.0,30,4.90
3187,20% OFF BLUE WAVE · 10min Beach & Airport!,92.0,22,4.95
3188,At Mine - Victoria King Suite with Parking,69.0,31,5.00
3189,Vintage Vibes,63.0,31,4.97


In [11]:
# 6. Get a list of properties with the following characteristics:
# * price < 150 OR more than one bathroom
# Use the "|" operator to implement the OR operator

listings_with_numeric_price = listings.withColumn("price_numeric", regexp_replace(col("price"), "[$,]", "").cast("double") )

result = (
    listings_with_numeric_price
    .where(
        (listings_with_numeric_price['price_numeric'] < 150) | 
        (listings_with_numeric_price['bathrooms'] > 1) 
        )
    .select(['name', 'price_numeric', 'bathrooms'])
    .dropna()
).toPandas()

result


,name,price_numeric,bathrooms
0,2 bd/2ba Oceanfront Condo,140.0,2.0
1,MIAMI - AMAZING APARTMENT OCEANVIEW,234.0,1.5
2,Tropical Beach Paradise Awaits You,119.0,1.0
3,Dual Masters! - Adult Media/Game Rm - 1GB Inte...,247.0,3.0
4,COSY APT OVER THE BEACH,250.0,1.5
...,...,...,...
10741,Chic Studio Near Amerant Arena & Shopping!,72.0,1.0
10742,"PS 1 Queen, TV, Minifrig",27.0,2.0
10743,Beautiful Apartment w/ Balcony 1BR- Near Beach,111.0,1.0
10744,Marvelous 3 bedroom house with 5ft Jacuzzi pool,202.0,2.0


In [12]:
# 7. Get the highest listing price in this dataset
# Consider using the "max" function from "pyspark.sql.functions"

result =  listings_with_numeric_price.agg(max("price_numeric"))
result.show()



+------------------+
|max(price_numeric)|
+------------------+
|           95000.0|
+------------------+



In [13]:
# 8. Get the name and a price of property with the highest price
# Try to use "collect" method to get the highest price first, and then use it in a "filter" call 
max_price =listings_with_numeric_price.agg(max("price_numeric")).collect()[0][0]

result = (
    listings_with_numeric_price
    .where(
        (listings_with_numeric_price['price_numeric'] == max_price) 
        )
    .select(['name', 'price_numeric', 'bedrooms','bathrooms'])
    .dropna()
).toPandas()

result

,name,price_numeric,bedrooms,bathrooms
0,Enjoy Peace & tranquillity. A home away from home,95000.0,2,2.0


In [14]:
# 9. Get the number of hosts in the dataset

result = listings.select("host_id").distinct().count()

print(f"{result} distinct hosts")

6440 distinct hosts


In [15]:
# check type of first_review column
listings.schema["first_review"].dataType # first_review is ???

DateType()

In [16]:
# 10. Get listings with a first review in 2024
# Consider using the "year" function from "pyspark.sql.functions"

result = (
    listings_with_numeric_price
    .where(
        (year(listings_with_numeric_price['first_review']) == 2024)
        )
    .select(['name', 'price_numeric', 'bedrooms','bathrooms', 'first_review', year(listings_with_numeric_price['first_review']).alias('first_review_year')])
    .dropna()
).toPandas()

result

,name,price_numeric,bedrooms,bathrooms,first_review,first_review_year
0,Sailboat Bend Retreat,280.0,3,2.5,2024-04-28,2024
1,Modern 2BR Home Free Parking,126.0,2,1.0,2024-12-09,2024
2,Nice Country Home in the city,232.0,3,3.0,2024-05-27,2024
3,Room # B Cozy Queen Bed with pool amenities,42.0,1,1.0,2024-03-12,2024
4,Steps to the Sand - Hollywood Beach Townhouse,165.0,4,4.0,2024-12-17,2024
...,...,...,...,...,...,...
2037,Cozy Private Room Steps from the Beach,90.0,1,1.0,2024-12-27,2024
2038,Sawgrass Mall - Newly Renovated,138.0,2,1.5,2024-12-29,2024
2039,"Comfortable & Affordable, Close to Everything!",82.0,2,1.0,2024-12-31,2024
2040,Blue Haven Vacations / Hot tub + Pool Table,257.0,4,2.0,2024-12-30,2024


In [17]:
# Stop Spark session
spark.stop()